In [1]:
import json
import subprocess #Need to clean up GPU memory after execution so forced to restart interpreter

In [2]:
scans = []
NUM_FIELD_BEGIN = 10
NUM_FIELD_END = 20

with open('working_dir/faber_sam1_s12_crack_tomo_TomoFastScan.dat') as dat_f:
    read_counter = 0
    cur_scan = {}

    for line in dat_f:
        if line.startswith('End'):
            read_counter = 2
        elif read_counter == 2:
            read_counter -= 1
            cur_scan['start'] = int(line.split(' ')[4].strip()) + NUM_FIELD_BEGIN
        elif read_counter == 1:
            read_counter -= 1
            cur_scan['end'] = int(line.split(' ')[4].strip()) - NUM_FIELD_END
            scans.append(cur_scan)
            cur_scan = {}

In [3]:
with open('working_dir/config.json', 'r') as config_f:
    config = json.load(config_f)

In [4]:
print(scans)

[{'start': 133706, 'end': 135506}, {'start': 135537, 'end': 137337}, {'start': 137368, 'end': 139168}, {'start': 139200, 'end': 141000}, {'start': 141031, 'end': 142831}, {'start': 142862, 'end': 144662}, {'start': 144694, 'end': 146494}, {'start': 146525, 'end': 148325}, {'start': 148356, 'end': 150156}, {'start': 150188, 'end': 151988}, {'start': 152019, 'end': 153819}, {'start': 153850, 'end': 155650}, {'start': 155682, 'end': 157482}, {'start': 157513, 'end': 159313}, {'start': 159344, 'end': 161144}, {'start': 161177, 'end': 162977}, {'start': 163008, 'end': 164808}, {'start': 164839, 'end': 166639}, {'start': 166670, 'end': 168470}, {'start': 168501, 'end': 170301}, {'start': 170332, 'end': 172132}, {'start': 172163, 'end': 173963}, {'start': 173994, 'end': 175794}]


In [5]:
for i, scan in enumerate(scans):
    assert scan['end'] - scan['start'] == len(config['omega'])
    config['img_range'] = [scan['start'], scan['end']]
    curr_config_fp = 'working_dir/temp_all_config.json'
    with open(curr_config_fp, 'w') as curr_conf_f:
        json.dump(config, curr_conf_f)
    subprocess.run(['python', 'coarse_mesh.py', f'{curr_config_fp}', 'working_dir/voids'])
    print(f'Finished scan {i}/{len(scans)}')

2022-09-26 09:10:31.706896: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-26 09:10:31.978726: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-09-26 09:10:34.708761: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-09-26 09:10:34.708890: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

	TIME: COARSE RECON 6.41 secs
	STAT: voids found - 27009
	TIME: LABELING 22.75 secs
void2mesh_mproc: preserve_feature : False, edge_thres : 0, b : 4, nprocs : None
